In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
#import tushare as ts
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torch.nn.functional as F
import scipy.io as scio
import time 

In [2]:
path_noisy = r'Input_Patches_2Dreal1.csv'

In [3]:
data = pd.read_csv(path_noisy, header=None)

In [4]:
#type(data.shape[1])

In [5]:
data = data.T

In [6]:
data

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.162368,0.219258,0.009214,0.198699,0.493524,0.245892,-0.182372,-0.156646,-0.316870,0.224624,...,0.038849,-0.073450,0.230653,0.186483,-0.103519,-0.055073,-0.205587,0.169509,-0.052408,-0.188393
1,-0.711692,-0.265535,0.404216,-0.017491,-0.240719,-0.671804,-0.229972,0.583406,0.595760,0.279998,...,0.407242,0.028367,-0.164915,-0.394898,-0.529706,-0.045514,-0.076049,-0.047949,0.080052,0.003718
2,-0.023045,-0.031333,0.190656,-0.153333,-0.318722,-0.327969,-0.852523,-0.108784,0.031883,0.301851,...,0.165481,-0.253374,-0.419997,-0.117057,-0.117169,-0.098420,0.363498,0.042636,0.243335,0.157551
3,-0.177922,-0.266030,-0.141247,0.487962,0.602046,0.296484,0.342437,-0.300288,-0.482048,0.086826,...,-0.239414,0.120151,0.163402,-0.273804,0.028751,0.531286,0.170631,0.200496,0.022241,-0.191563
4,0.176998,-0.079739,-0.257229,-0.217703,0.026047,0.226468,0.019902,-0.084082,0.066048,0.049374,...,0.048834,0.037723,0.456958,0.254121,0.387824,0.171179,0.337118,0.123482,-0.141251,-0.289645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27951,0.135708,-0.175223,-0.155916,0.110586,-0.101674,-0.305425,-0.304724,-0.350010,-0.216762,0.217116,...,-0.087081,0.034378,0.054869,-0.010448,-0.081313,-0.077376,0.238562,0.109800,0.283365,0.169007
27952,-0.106156,-0.003215,-0.538261,0.026924,-0.765845,-0.459111,-2.698122,-0.538279,-0.157002,-0.002314,...,0.105604,-0.352210,0.031722,0.339139,0.016016,-0.014568,-0.191193,0.067487,-0.060864,-0.162643
27953,0.048989,-0.056892,0.095801,-0.062812,-0.209297,0.077618,-0.220826,-0.194618,-0.324122,0.254190,...,-0.362360,0.030006,0.189344,-0.290240,0.434496,-0.103193,0.157770,0.663759,-0.059527,-0.308712
27954,0.075517,0.180355,0.128823,-0.116407,-0.018621,0.144102,-0.168198,-0.218335,-0.098015,0.132025,...,0.111506,0.284856,-0.038512,0.511344,0.274927,0.057396,0.040645,-0.143664,0.490805,0.276644


In [7]:
data = data.astype(np.float32)
data= torch.from_numpy(data.values)

In [8]:
data.shape

torch.Size([27956, 1024])

In [9]:
#将前80%作为训练集，后20%作为测试集
train_size = int(len(data) * 0.8)
train = data[:train_size]
vaild = data[train_size:]
print(train.shape)
print(vaild.shape)
batch_size1 = 64
w1 = 32
w2 = 32

train_data= TensorDataset(train)
vaild_data= TensorDataset(vaild)

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size = 64,
                                           shuffle = True)

vaild_loader = torch.utils.data.DataLoader(vaild_data,
                                          batch_size = 64,
                                          shuffle = True)

torch.Size([22364, 1024])
torch.Size([5592, 1024])


In [10]:
#定义Fully connected (FC) block
class FCB(nn.Module):
    def __init__(self, input_size, output_size, dropout=0.1):
        super().__init__()
        
        self.fc = nn.Linear(input_size, output_size)
        self.activation = nn.LeakyReLU()
        self.bn = nn.BatchNorm1d(output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc(x)
        x = self.activation(x) 
        x = self.bn(x)
        x = self.dropout(x)
        
        return x

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_size, dropout=0.1):
        super().__init__()


        self.fcb0 = FCB(input_size, 128, dropout)
        self.fcb1 = FCB(128, 64, dropout)

        self.fcb2 = FCB(64, 32, dropout)

        self.fcb3 = FCB(32, 16, dropout)

        self.fcb4 = FCB(16, 8, dropout)

        self.fcb5 = FCB(8, 4, dropout)

        self.fcb6 = FCB(4, 4, dropout)

        


    def forward(self, x):

        #x = x.reshape(x.shape[0],1,x.shape[1])

        x1 = self.fcb0(x)#x->128
        x2 = self.fcb1(x1)

        x3 = self.fcb2(x2)

        x4 = self.fcb3(x3)

        x5 = self.fcb4(x4)

        x6 = self.fcb5(x5)

        x7 = self.fcb6(x6)

        x8 = self.fcb6(x7)
        

        

        return x2,x3,x4,x5,x6,x7,x8

In [12]:
class Decoder(nn.Module):
    def __init__(self, output_size, dropout=0.1):
        super().__init__()
        self.pab1 = FCB(8, 8, dropout)
        self.pab2 = FCB(16, 16, dropout)
        self.pab3 = FCB(32, 32, dropout)
        self.pab4 = FCB(64, 64, dropout)
        self.pab5 = FCB(128, 128, dropout)
        self.pab6 = FCB(128, output_size, dropout)

        self.activation = nn.Identity()


    def forward(self,x2, x3,x4,x5,x6,x7,x8):
        
        x = torch.cat((x6,x8),dim=1)
        x = self.pab1(x)
        x = torch.cat((x,x5),dim=1)
        x = self.pab2(x)
        x = torch.cat((x,x4),dim=1)
        x = self.pab3(x)
        x = torch.cat((x,x3),dim=1)
        x = self.pab4(x)
        x = torch.cat((x,x2),dim=1)
        x = self.pab5(x)
        x = self.pab6(x)

        x = self.activation(x)
        return x

In [13]:
class AutoEncoder(nn.Module):
    def __init__(self,input_size, output_size):
        super().__init__()
        self.encoder = Encoder(input_size)
        self.decoder = Decoder(output_size)

    def forward(self, x):
        x2, x3,x4,x5,x6,x7,x8= self.encoder(x)
        x = self.decoder(x2, x3,x4,x5,x6,x7,x8)

        return x

In [14]:
device = torch.device("cuda") 
model = AutoEncoder(w1*w2,w1*w2).to(device)
#将模型转移到GPU上
#criterion = MeanHuberLoss(delta=1.3)
#criterion = WelschLoss(delta=0.5)
#criterion = Loss0(delta=0.46,r=0.05)#0.5 and 0.2,SNR:-8dB ok
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [15]:
start_time = time.time()
es_cnt = 0
es_thres = 5
prev_train_loss = float('inf')
loss_train = []
loss_vaild = []
num_epochs = 100 # 总训练轮数
#num_batch_train = 0
for epoch in range(num_epochs):
  #train_bar = tqdm(train_loader)
  train_loss = 0.0
  
  for i , (batch) in enumerate(train_loader):

    # 数据转到device
    train_batch = batch[0].to(device)
    
    # 训练步骤  
    optimizer.zero_grad()
    outputs = model(train_batch)
    loss = criterion(outputs, train_batch)
    loss.backward() 
    optimizer.step()
    
    train_loss += loss.item()
    #num_batch_train +=1
  #train_loss除以所有bacth个数
  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
  loss_train.append(train_loss)
    



  # 验证
  valid_loss = 0.0
  #num_batch_vaild = 0
  with torch.no_grad():
    for i , (batch) in enumerate(vaild_loader):
    #for batch in vaild_loader:
    
      val_batch = batch[0].to(device)
      
      outputs = model(val_batch)
      loss = criterion(outputs, val_batch)
      valid_loss += loss.item()
      #num_batch_vaild += 1
    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
    loss_vaild.append(valid_loss)
    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))

    
    # Early stopping
    if train_loss - prev_train_loss >= 0:
        es_cnt += 1
    else:
        #es_cnt = 0
        pass

    if es_cnt >= es_thres:
        print(f"Early stopped at epoch {epoch}, train loss stop improving")
        break  

    prev_train_loss = train_loss
  print('loss_train: ', loss_train)
  print('vaild_train: ',loss_vaild)          
print("Training finished")
current_time = time.time()
time_sum = current_time-start_time
print(time_sum)

Epoch [1/100], Train Loss: 0.8045, Valid Loss: 0.5527
loss_train:  [0.8044639311517988]
vaild_train:  [0.5526764386079528]
Epoch [2/100], Train Loss: 0.3638, Valid Loss: 0.2724
loss_train:  [0.8044639311517988, 0.3638168154018266]
vaild_train:  [0.5526764386079528, 0.27239029007879173]
Epoch [3/100], Train Loss: 0.1957, Valid Loss: 0.1605
loss_train:  [0.8044639311517988, 0.3638168154018266, 0.1956858733722142]
vaild_train:  [0.5526764386079528, 0.27239029007879173, 0.16050851378928532]
Epoch [4/100], Train Loss: 0.1444, Valid Loss: 0.1235
loss_train:  [0.8044639311517988, 0.3638168154018266, 0.1956858733722142, 0.1444166673081262]
vaild_train:  [0.5526764386079528, 0.27239029007879173, 0.16050851378928532, 0.12354262731969357]
Epoch [5/100], Train Loss: 0.1323, Valid Loss: 0.1125
loss_train:  [0.8044639311517988, 0.3638168154018266, 0.1956858733722142, 0.1444166673081262, 0.1322598226581301]
vaild_train:  [0.5526764386079528, 0.27239029007879173, 0.16050851378928532, 0.123542627319693

In [16]:
loss_train = pd.DataFrame(loss_train)
loss_vaild = pd.DataFrame(loss_vaild)

loss = pd.concat([loss_train,loss_vaild],axis=1)

In [17]:
loss.columns = ['train_loss','vaild_loss']

In [18]:
torch.save(model.state_dict(), r'.\model_2dreal1patch.pth')

In [19]:
model = AutoEncoder(w1*w2,w1*w2).to(device)
data = data.to(device)
model.load_state_dict(torch.load(r'.\model_2dreal1patch.pth'))
model.eval()
with torch.no_grad():
    output = model(data)
    #loss = criterion(output, data)
print(output.shape)

torch.Size([27956, 1024])


In [20]:
output = output.cpu()
output = output.numpy()
output = pd.DataFrame(output)

In [21]:
loss.to_csv(r'loss_2dreal1patch.csv',index=False)

In [22]:
output.to_csv(r'./output_2dreal1patch.csv',index=None,header=None)

In [23]:
#loss_0 = []
#start_time = time.time()
#es_cnt = 0
#es_thres = 5
#prev_train_loss = float('inf')
#loss_train = []
#loss_vaild = []
#num_epochs = 100 # 总训练轮数
##num_batch_train = 0
#for epoch in range(num_epochs):
#  #train_bar = tqdm(train_loader)
#  train_loss = 0.0
#  
#  for i , (batch) in enumerate(train_loader):
#
#    # 数据转到device
#    train_batch = batch[0].to(device)
#    
#    # 训练步骤  
#    optimizer.zero_grad()
#    outputs = model(train_batch)
#    loss = criterion(outputs, train_batch)
#    loss.backward() 
#    optimizer.step()
#
#    loss_0.append(loss.item())
#
#    train_loss += loss.item()
#    #num_batch_train +=1
#  #train_loss除以所有bacth个数
#  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
#  loss_train.append(train_loss)
#    
#
#
#
#  # 验证
#  valid_loss = 0.0
#  #num_batch_vaild = 0
#  with torch.no_grad():
#    for i , (batch) in enumerate(vaild_loader):
#    #for batch in vaild_loader:
#    
#      val_batch = batch[0].to(device)
#      
#      outputs = model(val_batch)
#      loss = criterion(outputs, val_batch)
#      valid_loss += loss.item()
#      #num_batch_vaild += 1
#    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
#    loss_vaild.append(valid_loss)
#    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))
#
#    
#    # Early stopping
#    if train_loss - prev_train_loss >= 0:
#        es_cnt += 1
#    else:
#        #es_cnt = 0
#        pass
#
#    if es_cnt >= es_thres:
#        print(f"Early stopped at epoch {epoch}, train loss stop improving")
#        break  
#    
#
#
##   if epoch == 10:
##     torch.save(model.state_dict(), r'.\model_epochs10.pth')
##   elif epoch == 20:
##     torch.save(model.state_dict(), r'.\model_epochs20.pth')
##   elif epoch == 30:
##     torch.save(model.state_dict(), r'.\model_epochs30.pth')
##   elif epoch == 40:
##     torch.save(model.state_dict(), r'.\model_epochs40.pth')
##   elif epoch == 50:
##     torch.save(model.state_dict(), r'.\model_epochs50.pth')
##   elif epoch == 60:
##     torch.save(model.state_dict(), r'.\model_epochs60.pth')
##   elif epoch == 70:
##     torch.save(model.state_dict(), r'.\model_epochs70.pth')
##   elif epoch == 80:
##     torch.save(model.state_dict(), r'.\model_epochs80.pth')
##   elif epoch == 90:
##     torch.save(model.state_dict(), r'.\model_epochs90.pth')
##   elif epoch == 100:
##     torch.save(model.state_dict(), r'.\model_epochs100.pth')
##   else:
##       pass
#
#    prev_train_loss = train_loss
#  print('loss_train: ', loss_train)
#  print('vaild_train: ',loss_vaild)          
#print("Training finished")
#current_time = time.time()
#time_sum = current_time-start_time
#print(time_sum)